In [1]:
import sys
import json
import pandas as pd
import langchain
import numpy as np

from typing import Optional
from google.cloud import aiplatform
from langchain.document_loaders import ConfluenceLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

from vertexai.language_models import TextEmbeddingModel

sys.path.append('../')

from src.upload_data.config import (
    CONFLUENCE_URL
    CONFLUENCE_API_KEY,
    CONFLUENCE_SPACE_NAMES,
    CONFLUENCE_USERNAME,
    EMBEDDING_PATH,
    PROJECT_ID,
    REGION,
    BUCKET_NAME
)

In [2]:
def load_from_confluence_loader(
    confluence_url=CONFLUENCE_URL
    username=CONFLUENCE_USERNAME,
    api_key=CONFLUENCE_API_KEY,
    space_key=CONFLUENCE_SPACE_NAMES,
):
    """Load HTML files from Confluence"""
    loader = ConfluenceLoader(
        url=confluence_url,
        username=username,
        api_key=api_key
    )

    docs = loader.load(
        space_key=space_key,
        max_pages=50,
        keep_markdown_format=True
    )

    return docs

In [3]:
docs = load_from_confluence_loader()

In [14]:
docs[1]

Document(page_content='### **MON MATÉRIEL PHOTO 📷 :**\n\n* Un mur blanc ou un drap blanc (tendre le drap si il n\'y a pas de mur blanc)\n* Un appareil photo ou un téléphone\n* Un trépied ou un super photographe (on peut mettre, mari, femme, parents, voisins, enfants etc.. à contribution avec une petite boite de chocolat ça passe bien)\n* Cadrage buste entier.\n\n### **ÉTAPE 1: La mise en place de votre “studio photo”.**\n\n* Trouvez un mur blanc, de préférence dans une pièce qui reçoit beaucoup de lumière du jour. Si vous n’avez pas de mur blanc, ou si celui que vous avez est recouvert de photos, accrochez un grand drap blanc au plafond et faites-le pendre jusqu’au sol.\n* Ouvrez les volets et laissez la lumière du soleil inonder la pièce. Organisez-vous pour commencer votre séance photo lorsque vous savez que vous allez recevoir beaucoup de lumière du soleil dans cette pièce.\n* Procurez-vous des lampes avec un abat-jour fermé. Servez-vous des lampes pour remplir la pièce de lumière d

In [15]:
def splitter(docs):
    # Markdown
    headers_to_split_on = [
        ("#", "Titre 1"),
        ("##", "Sous-titre 1"),
        ("###", "Sous-titre 2"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

    # Split based on markdown and add original metadata
    md_docs = []
    for doc in docs:
        md_doc = markdown_splitter.split_text(doc.page_content)
        for i in range(len(md_doc)):
            md_doc[i].metadata = md_doc[i].metadata | doc.metadata
        md_docs.extend(md_doc)

    # RecursiveTextSplitter
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    # Chunk size big enough
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
        separators=["\n\n", "\n", "(?<=\. )", " ", ""]
    )

    splitted_docs = splitter.split_documents(md_docs)

    return splitted_docs

In [16]:
splitted_docs = splitter(docs)
splitted_docs[0]

Document(page_content='Saluez vos collègues qui souhaitent connaître votre nom, vos pronoms, votre rôle, votre équipe et votre emplacement (ou si vous télétravaillez).', metadata={'title': "Vue d'ensemble", 'id': '98394', 'source': 'https://florianbastin.atlassian.net/wiki/spaces/~70121e1c1cf2b203a49dabc6762c43bdbfe05/overview'})

In [17]:
def init_sample(
    project: Optional[str] = None,
    location: Optional[str] = None,
    experiment: Optional[str] = None,
    staging_bucket: Optional[str] = None,
    credentials = None,
    encryption_spec_key_name: Optional[str] = None,
    service_account: Optional[str] = None,
):

    from google.cloud import aiplatform

    aiplatform.init(
        project=project,
        location=location,
        experiment=experiment,
        staging_bucket=staging_bucket,
        credentials=credentials,
        encryption_spec_key_name=encryption_spec_key_name,
        service_account=service_account,
    )

In [18]:
from google.cloud import aiplatform

aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=f"gs://{BUCKET_NAME}"
)

In [19]:
model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")


In [20]:
import time
EMBEDDING_PATH = '../data/embeddings.json'
with open(EMBEDDING_PATH, "w") as f:
    time_st = time.time()
    for doc_id, splitted_doc in enumerate(splitted_docs):
        try:
            embeddings = model.get_embeddings([splitted_doc.page_content])
        except:
            sleeping_quota_time = 70 - (time.time() - time_st)
            print(f"sleeping for {sleeping_quota_time} because of TextEmbedding quotas")
            time.sleep(sleeping_quota_time) # sleep because max of X requests per minutes
            time_st = time.time()
            embeddings = model.get_embeddings([splitted_doc.page_content])

        id_embeddings = {
            "id": doc_id,
            "embedding": [float(x) for x in list(embeddings[0].values)]
        }
        json.dump(id_embeddings, f)
        f.write("\n")

In [38]:
# Endpoint

In [39]:
len(embeddings[0].values)

768

In [40]:
index_endpoint_id = "6440411349930475520"
index_id = "7038053094231375872"

In [41]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
    index_endpoint_name=index_endpoint_id
)

In [42]:
question = "Comment accéder à mon comité d'entreprise ?"
question_emb = model.get_embeddings([question])

In [43]:
# Find neighbors

In [44]:
neighbors = my_index_endpoint.find_neighbors(
            deployed_index_id="basfdeployedindex",
            queries=[np.array(question_emb[0].values)],
            num_neighbors=5
        )


In [45]:
neighbors

[[MatchNeighbor(id='105', distance=0.8089103698730469),
  MatchNeighbor(id='107', distance=0.7539622783660889),
  MatchNeighbor(id='111', distance=0.7379288673400879),
  MatchNeighbor(id='46', distance=0.7317922711372375),
  MatchNeighbor(id='106', distance=0.7313551902770996)]]

In [46]:
for neighbor in neighbors[0]:
    doc_id = neighbor.id
    print(splitted_docs[int(doc_id)])

page_content='Pour accéder au CE, vous pouvez consultant l’adresse suivante: <https://mon-CE.fr>  \nSi vous n’avez pas vos identifiants, vous pouvez envoyer un mail à [xxxx@mon-ce.fr](mailto:xxxx@mon-ce.fr), le responsable du comité d’entreprise.  \nRenseignez les identifiants sur le site du CE et vous pourrez bénéficier de nombreux avantages.' metadata={'Sous-titre 1': 'Comment accéder à mon CE ?', 'title': "Comité d'entreprise (CE) - Définition et rôles", 'id': '1933313', 'source': 'https://florianbastin.atlassian.net/wiki/spaces/~70121e1c1cf2b203a49dabc6762c43bdbfe05/pages/1933313'}
page_content="Le Comité d'Entreprise est chargé de veiller au respect des droits des salariés en matière de travail, de sécurité, de santé et de conditions de travail. Il peut être consulté par la direction de l'entreprise sur différentes questions liées à l'organisation du travail, aux licenciements collectifs, aux restructurations, etc. Il est également informé des projets de l'entreprise et peut émett

In [ ]:
# Firestore


In [22]:
from tqdm import tqdm

import firebase_admin
from firebase_admin import firestore

In [ ]:
from google.cloud import firestore
import uuid

from google.cloud import datastore
db = datastore.Client(project=PROJECT_ID)

In [ ]:
PROJECT_ID

In [32]:
#app = firebase_admin.initialize_app()
db = firestore.Client(project=PROJECT_ID, database="basf-rag")

In [33]:
# Create Agent

In [34]:
from typing import Any, Dict, List, Optional, Union

import numpy as np
import firebase_admin
from firebase_admin import firestore
from google.cloud import aiplatform
from langchain.schema import BaseRetriever, Document
from langchain.callbacks.manager import CallbackManagerForRetrieverRun, AsyncCallbackManagerForRetrieverRun
from langchain.embeddings.base import Embeddings
from langchain.embeddings import TensorflowHubEmbeddings

In [35]:
def make_doc(doc):
    metadata_keys_to_add = [
        'source',
        'title',
        'Titre 1',
        'Sous-titre 1',
        'Sous-titre 2'
    ]

    answers = {
        "content": doc.page_content
    }

    for key in doc.metadata.keys():
        if key in metadata_keys_to_add:
            answers[key] = doc.metadata[key]

    return answers

In [37]:
# for doc_id, splitted_doc in enumerate(splitted_docs):
#     doc = make_doc(splitted_doc)
#     db.collection("basf-rag").document(str(doc_id)).set(doc)

In [47]:
class FirestoreRetriever(BaseRetriever):
    index_endpoint_name: str
    deployed_index_id: str
    embeddings: Embeddings
    collection: str
    top_k: int = 5

    def _similarity_search(self, query_emb: np.ndarray):
        """
        Perform a similarity search.

        Args:
            query_emb: Query represented as an embedding

        Returns:
            A list of documents most similar to the query
        """
        my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
            index_endpoint_name=self.index_endpoint_name
        )

        similar_docs = my_index_endpoint.find_neighbors(
            deployed_index_id=self.deployed_index_id,
            queries=query_emb,
            num_neighbors=self.top_k
        )

        return similar_docs

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        query_embedding = self.embeddings.embed_documents([query])
        similar_docs = self._similarity_search(query_embedding)

        relevant_docs = []
        for doc in similar_docs[0]:
            doc_id = doc.id
            doc_ref = db.collection(self.collection).document(doc_id)

            doc = doc_ref.get()
            relevant_docs.append(self._firestore_doc_to_langchain_doc(doc))
        return relevant_docs


    async def _aget_relevant_documents(
        self, query: str, *, run_manager: AsyncCallbackManagerForRetrieverRun
    ) -> List[Document]:
        raise NotImplementedError()

    def _firestore_doc_to_langchain_doc(self, fs_doc) -> Document:
        lc_doc = Document(
            page_content=fs_doc.get("content")
        )
        return lc_doc

In [50]:
# Lanchain Embedding

In [52]:
from langchain.embeddings.vertexai import VertexAIEmbeddings
embeddings = VertexAIEmbeddings(project=PROJECT_ID)

In [65]:
from langchain.prompts import PromptTemplate
def get_template():
    template = """
    Given this text extracts:
    -----
    {context}
    -----
    Please anwser in French and start with "Bonjour Monsieur".
    The question is:
    Question: {question}
    Helpful Answer:
    """
    return template

def get_prompt() -> PromptTemplate:
    prompt = PromptTemplate(
        template=get_template(),
        input_variables=["context", "question"]
    )
    return prompt

In [67]:
chain_type_kwargs = {"prompt": get_prompt()}


In [49]:
# Agent

In [75]:
index_endpoint_id = "6440411349930475520"
index_id = "7038053094231375872"


retriever = FirestoreRetriever(
    index_endpoint_name=index_endpoint_id,
    deployed_index_id="basfdeployedindex",
    collection="basf-rag",
    embeddings = embeddings,
    top_k=5
)

In [76]:
query = "when was the college of engineering in the University of Notre Dame established?"
query = "In what year did the initial degrees get handed out at Notre Dame?"

In [77]:
from langchain.chat_models import ChatVertexAI
from langchain.chains import RetrievalQA

In [78]:
llm = ChatVertexAI(location="europe-west1")

In [79]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs
)

In [81]:
query = "Comment accéder à mon comité d'entreprise ? "
print(qa.run(query))

 Bonjour Monsieur,

Pour accéder à votre comité d'entreprise, vous pouvez consulter l'adresse suivante : <https://mon-CE.fr>. 
Si vous n'avez pas vos identifiants, vous pouvez envoyer un mail à [xxxx@mon-ce.fr](mailto:xxxx@mon-ce.fr), le responsable du comité d'entreprise. 
Renseignez les identifiants sur le site du CE et vous pourrez bénéficier de nombreux avantages.
